## Van Hove Function Tutorial

This tutorial will go over the calculation of the Van Hove Function (VHF) using the `scattering` package.  The VHF is defined as the time-dependent radial distribution function (RDF), $G(r, t)$ where $r$ is distance and $t$ is time.  That is, the probability of finding particle $j$ at $t=t$ at a distance $r$ from the position of particle $i$ at $t=0$.  The VHF analysis is useful as we can study the correlations between particles over time.

The VHF is defined as the following for fluids containing monotonic atoms:
\begin{equation}
     G(r,t) = \frac{1}{4\pi\rho N r^2}\sum_{i, j} \delta(r - |\boldsymbol{r_i    }(0) - \boldsymbol{r_j}(t)|)
\end{equation}

where $\rho$ is the average number density of atoms, $N$ is the number of atoms in the system, $r_j(t)$ is the position of the $j$th atom at time $t$ and $\delta(r)$ is the Dirac delta function.

For fluids of polyatomic molecules, the VHF is computed as a summation of the VHFs between atoms $\alpha$ and $\beta$, which are referred to the partial VHFs.  This equation for a partial VHF is defined as:

\begin{equation}
 G_{\alpha\beta}(r,t)  = \frac{V}{4\pi N_\alpha N_\beta r^2}
\sum_{i\in\{\alpha\}}\sum_{i\in\{\beta\}} \delta(r - |\boldsymbol{r_i}(0    ) -
 \boldsymbol{r_j}(t)|)
 \end{equation}
 
 The total VHF for a fluid computed from the partial VHFs is then defined as:
 
\begin{equation}
\begin{gathered}
G(r,t) = \sum_{\alpha=1}^{N_\alpha} \sum_{\beta=1}^{N_\beta} x_{\alpha}     x_{\beta} \bar{b}_{\alpha} \bar{b}_{\beta}G_{\alpha\beta}(r,t) \\
\end{gathered}
\end{equation}

where $x_{\beta}$ is the atomic fraction for species $\beta$,and $\bar{b}_{\alpha}$ is the form factor proportional to the number of electrons in species $\alpha$.

### The 'scattering' package
`scattering` uses `MDTraj` as the backend to load in simulation trajectories for computing the VHF.  There are two main functions for computing the VHF.  Distance calculations are computed with underlying C++ code for computational efficiency.

The partial VHF for two species can be computed with the function `van_hove.compute_partial_van_hove`.  The total VHF for a polyatomic fluid can be computed with the function `van_hove.compute_van_hove`.  This tutorial will cover both functions below.

### Installation Instructions
`scattering` requires the following packages:
- numpy
- mdtrj
- scipy
- progressbar2
- cython

At the time of creating this notebook, the distance calculations for the VHF have not been officially released in MDTraj through Conda.  As a result, MDTraj will need to be installed from source on GitHub.  The following steps are recommended to get a minimum working environment for this example:

- `conda create -n vhf python=3.7`
- `conda install -c conda-forge -c anaconda numpy scipy jupyter progessbar2 cython`
- `git clone https://github.com/mdtraj/mdtraj` Note: It's recommended this repository is installed in a separate directory to keep track of versions specific to this tutorial.
- `cd mdtraj` and `pip install -e .`
- `cd ..` (cd out of the `mdtraj` repository)
- `git clone https://github.com/mattwthompson/scattering.git`
- `cd scattering` and `pip install -e .`

### MDTraj basics

Before covering the VHF analysis functions, a brief overview of `MDTraj` is given below.

### MDTraj `trajectory` object.
The main data structure in `MDTraj` is the `trajectory` object, which contains the position information for all atoms and all frames of a simulation.

A `trajectory` can be initialized with the `md.load()` function.

In [ ]:
# Import MDtraj as alias "md"
import mdtraj as md

# Load in a water simulation as a trajectory
trj = md.load("../scattering/tests/files/spce.xtc", top="../scattering/tests/files/spce.gro")

trj

The `repr` displays important information regarding the simulation trajectory.  `trj` contains 1001 frames, 1050 atoms, and 350 residues.

The `dir` function can be used to view the methods and attributes available within the `trajectory` class

In [ ]:
dir(trj)

For example, we can view information regarding the box and positions of atoms

In [ ]:
# Get positions of the atoms
trj.xyz

In [ ]:
# Get unitcell lengths
trj.unitcell_lengths

#### `Topology` class
The `Topology` class in `MDTraj` contains the topology information of your chemical system, including atoms, residues, and bonds.

If `top` is passed as an argument when loading in a `trajectory`, the `topology` can be accessed by calling `trajectory.topology`.

In [ ]:
trj.topology

The `topology` contains a `select` method to select specific atoms from your trajectory using a specific atom selection language.  See: https://mdtraj.org/1.9.4/atom_selection.html for reference.

Below we will select all oxygen atoms in the trajectory.

In [ ]:
trj.topology.select("name O")

A subtrajectory of specific atoms can be initialized by calling the `atom_slice` method.

In [ ]:
oxygens = trj.atom_slice(trj.topology.select("name O"))

oxygens

## Computing the Partial Van Hove Function
First, we will compute the partial VHF for specific atom species in our simulation trajectory.  The function within the package is `van_hove.compute_partial_van_hove`.  Below the docstring for the function is displayed.

In [ ]:
from scattering import van_hove

?van_hove.compute_partial_van_hove

Let's go ahead and calculate the partial VHF between the oxygen atoms in our system.  In this case, `selection1` and `selection2` will be `"name O"`.  For demonstration purposes, we will select the chunk_length to be `20`, or 20 ps.

In [ ]:
chunk_length = 20
r, g_r_t = van_hove.compute_partial_van_hove(trj=trj,
                                            chunk_length=chunk_length,
                                            selection1="name O",
                                            selection2="name O",)

In [ ]:
# plot the result with Matplotlib
import matplotlib.pyplot as plt

In [ ]:
t = trj.time[:chunk_length+1]
for j in range(chunk_length):
    plt.plot(r, g_r_t[j], label='{:.3f} ps'.format(t[j]))

plt.xlim((0, 0.8))
plt.ylim((0, 3.5))

plt.xlabel("position, r, nm")
plt.ylabel("g(r, t)")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

The data is quite noisy as this is a fairly small system with a small number of frames.  Noise will be reduced with longer trajectories and/or better averaging.

## Exercise 1
- Calculate and plot g(r, t) for the oxygen-hydrogen and hydrogen-hydrogen pairs.

## Computing the Total Van Hove Function
Next, the total VHF for a system of water will be computed using the function `van_hove.compute_van_hove`.  The documentation for the function is shown below.

This functions calculates the partial VHFs (`van_hove.compute_partial_van_hove`) for each pairwise interaction and them sums them using the equation above for polyatomic fluids.

In [ ]:
?van_hove.compute_van_hove

### Determining form factor for each partial VHF
As shown above, the form factor $\bar{b}_{\alpha}$ is related to the electron count for each atomic species.  By default, the form factor is estimated as the atomic number for each atom when `water=False`.  When `water=True`, specific form factors for the oxygen and hydrogen atoms of water will be specified as defined from the paper: https://pubmed.ncbi.nlm.nih.gov/29291242/

Let's go ahead and calculate the total VHF for our system of water.  We will specify `water=True` to specify the specific form factors for the oxygen and hydrogen atoms.

In [ ]:
chunk_length = 20
r, t, g_r_t = van_hove.compute_van_hove(trj=trj,
                                            chunk_length=chunk_length,
                                            water=True,)

In [ ]:
for j in range(chunk_length):
    plt.plot(r, g_r_t[j], label='{:.3f} ps'.format(t[j]))

plt.xlim((0, 0.8))
plt.ylim((0, 3.5))

plt.xlabel("position, r, nm")
plt.ylabel("g(r, t)")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

You may notice that the total VHF looks quite similar to the partial VHF between the oxygen atoms.  The reason for this is that the form factor for oxygen is $9\frac{1}{3}$ while the form factor is $\frac{1}{3}$.  Thus, the oxygen interactions dominate the total VHF.

## Exercise 2
- Write your own code to compute the total VHF from the three partial VHFs.  You will know you've reached the correct solution when your total VHF roughly equals the total VHF above.